In [62]:
import pandas as pd
import numpy as np
import scipy.io
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

### Data loading and preprocessing

In [63]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'
    

In [64]:
#here we use these columns nly to get the subject ID
foof = pd.read_csv(DATA_DIR+"foof2features.csv")
foof = foof.rename(columns={"C1": "IDs" ,"C2": "Intercept", "C3": "Slope"})
foof

,IDs,Intercept,Slope
0,NDARAA075AMK,0.986272,1.825774
1,NDARAA112DMH,1.486650,1.888544
2,NDARAA117NEJ,1.593155,2.095749
3,NDARAA947ZG5,0.703331,1.724831
4,NDARAA948VFH,0.918020,1.749441
...,...,...,...
2037,NDARZN277NR6,1.351549,1.996940
2038,NDARZN578YDP,1.380795,2.036327
2039,NDARZN610GTY,0.339229,1.050644
2040,NDARZN677EYE,0.781225,1.470061


In [65]:
data = scipy.io.loadmat(DATA_DIR+'x_nofoof.mat')  
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
columns = np.asarray([['Electrode %d - %d/2 Hz'%(i+1, j+1)] for i in range(data['x'].shape[1]) for j in range(data['x'].shape[2])])
df.columns = columns
df['IDs'] = foof['IDs']
df



,"(Electrode 1 - 1/2 Hz,)","(Electrode 1 - 2/2 Hz,)","(Electrode 1 - 3/2 Hz,)","(Electrode 1 - 4/2 Hz,)","(Electrode 1 - 5/2 Hz,)","(Electrode 1 - 6/2 Hz,)","(Electrode 1 - 7/2 Hz,)","(Electrode 1 - 8/2 Hz,)","(Electrode 1 - 9/2 Hz,)","(Electrode 1 - 10/2 Hz,)",...,"(Electrode 105 - 71/2 Hz,)","(Electrode 105 - 72/2 Hz,)","(Electrode 105 - 73/2 Hz,)","(Electrode 105 - 74/2 Hz,)","(Electrode 105 - 75/2 Hz,)","(Electrode 105 - 76/2 Hz,)","(Electrode 105 - 77/2 Hz,)","(Electrode 105 - 78/2 Hz,)","(Electrode 105 - 79/2 Hz,)",IDs
0,22.646692,21.409617,11.791191,7.654361,5.294041,4.281935,3.556075,2.891063,2.375165,2.321368,...,0.014442,0.013248,0.011966,0.011954,0.011685,0.012114,0.011274,0.011133,0.009402,NDARAA075AMK
1,35.803485,31.315928,20.393061,13.173540,6.198943,4.636760,4.205091,4.182367,3.032023,2.775398,...,0.029482,0.035029,0.032905,0.029361,0.025549,0.021678,0.023353,0.021711,0.023934,NDARAA112DMH
2,13.719575,13.953344,8.733713,5.339339,3.654114,3.280150,3.597552,3.324361,2.615713,1.973499,...,0.021167,0.019864,0.018397,0.016645,0.015293,0.015262,0.013120,0.013712,0.014215,NDARAA117NEJ
3,7.091286,7.064044,3.388819,2.020668,1.283089,1.225795,1.391197,1.194422,1.073312,0.945446,...,0.012450,0.014467,0.014399,0.014101,0.012216,0.012885,0.011950,0.011479,0.011285,NDARAA947ZG5
4,9.878377,9.354415,5.324718,3.351567,2.634583,2.559906,2.273353,1.941904,1.331602,0.881906,...,0.009977,0.010225,0.009637,0.009370,0.008455,0.008211,0.008064,0.008207,0.008642,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,2.089436,2.277007,1.587069,1.361913,1.177022,0.991350,0.730675,0.536954,0.399566,0.340996,...,0.014905,0.014557,0.014418,0.012811,0.012592,0.012685,0.013041,0.012496,0.011033,NaN
2052,10.152438,9.639543,5.917194,3.909757,2.970778,2.726462,2.339991,2.024998,1.875718,1.679626,...,0.014360,0.012237,0.011046,0.011192,0.011132,0.011145,0.010181,0.010405,0.011229,NaN
2053,3.654829,3.047049,1.688567,1.394472,1.627901,1.574334,1.852648,1.315756,1.039766,1.150138,...,0.051279,0.052121,0.051111,0.051777,0.052505,0.048095,0.049762,0.050901,0.049706,NaN
2054,4.242865,4.473603,3.151083,2.222441,1.890155,2.017281,2.327400,2.495586,2.321074,2.505722,...,0.027194,0.025103,0.022879,0.021707,0.020007,0.021364,0.020383,0.018960,0.018877,NaN


In [66]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
print('Before:'+str(beh.shape))

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'No Diagnosis Given', 'Communication Disorder',
                         'Depressive Disorders']

# most_common_disorders = ['Other Neurodevelopmental Disorders', 'ADHD-Inattentive Type', 'ADHD-Combined Type', 'Anxiety Disorders', 'No Diagnosis Given', 'Depressive Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# find users that have no diagnosis within these top diseases
# filtering should cahnge anything as this should also happen at a later stage
mask = None
for col in category_columns:
    mask_col = beh[col].isin(most_common_disorders)
    if mask is None:
        mask = mask_col
    else:
        mask = mask | mask_col

initial_size = beh.shape[0]
beh = beh[mask]
beh = beh.reset_index(drop=True)
new_size = beh.shape[0]
print('After:'+str(beh.shape))
print('Removing', initial_size - new_size,
      'patients as their diagnoses were very uncommon.')

Before:(3076, 177)
After:(2813, 177)
Removing 263 patients as their diagnoses were very uncommon.


In [67]:
no_diagnosis_given = 'No Diagnosis Given'

if no_diagnosis_given in most_common_disorders:
    no_diag_index = most_common_disorders.index(no_diagnosis_given)
    most_common_disorders = most_common_disorders[:no_diag_index] + \
        most_common_disorders[no_diag_index + 1:]

diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}
diagnoses_to_ids

{'Attention-Deficit/Hyperactivity Disorder': 0,
 'Anxiety Disorders': 1,
 'Specific Learning Disorder': 2,
 'Autism Spectrum Disorder': 3,
 'Disruptive': 4,
 'Communication Disorder': 5,
 'Depressive Disorders': 6}

In [68]:
def get_disorder(data, row, index):
    disorder = data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(beh.shape[0]):
    i = 0
    disorder = get_disorder(beh, k, i)
    disorders_patient = []
    while disorder != no_diagnosis_given and not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(beh, k, i)

    order_of_disorders.append(disorders_patient)


In [69]:
max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

classes = np.zeros((len(most_common_disorders),
                    beh.shape[0]), dtype=np.int32)

df_disorders = beh[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)

    disorder_df = df_disorders[mask.any(axis=1)]

    np.add.at(classes[i], disorder_df.index.values, 1)

behaviour_data_columns = beh.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = beh.drop(columns=columns_to_drop)

for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)


In [70]:
labels=behaviour_data[["IDs"]+list(most_common_disorders)]
labels

,IDs,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders
0,NDARAA075AMK,0,0,0,0,0,0,0
1,NDARAA112DMH,1,0,0,0,1,0,0
2,NDARAA117NEJ,1,0,0,0,1,0,0
3,NDARAA306NT2,1,1,1,0,0,1,0
4,NDARAA504CRN,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2808,NDARZZ007YMP,0,0,0,1,0,0,0
2809,NDARZZ740MLM,1,0,0,0,0,0,0
2810,NDARZZ810LVF,0,0,0,1,0,1,0
2811,NDARZZ830JM7,0,0,0,1,0,0,0


In [71]:

df = pd.merge(df, labels, on='IDs', how='inner')
df.shape



(1836, 8303)

In [72]:

#removing NaNs
df = df.dropna()
df.shape

(1823, 8303)

### Data Split

In [73]:
x = df[df.columns.difference(['IDs']+most_common_disorders)]
y = df[most_common_disorders]

# summarize dataset shape
print(x.shape, y.shape)

(1823, 8295) (1823, 7)


In [74]:
#scaling features

# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x)

# transform training data
x_norm = norm.transform(x)
x_norm = x


/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


In [75]:


train_features, test_features, train_labels, test_labels = train_test_split(x, y, test_size=0.3, shuffle=True)

### Metrics

In [76]:
from sklearn.metrics import hamming_loss, accuracy_score
import sklearn.metrics as skm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np


def evaluate(y_test, y_pred):
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Hamming Loss:", hamming_loss(y_test, y_pred))
    print("Classification Report:\n", skm.classification_report(y_test,y_pred, zero_division=1))
    print("Confusion matrix:\n", skm.multilabel_confusion_matrix(y_test, y_pred))

## Models

### Multi Output Classifier

In [77]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC as svm
from sklearn.multioutput import MultiOutputClassifier

forest = RandomForestClassifier(random_state=1)
lg = LogisticRegression()
svm = svm()
models = [lg, forest, svm]

for model in models:

    multi_output_model = MultiOutputClassifier(model, n_jobs=-1)
    multi_output_model.fit(train_features, train_labels)
    predicted_labels = multi_output_model.predict(test_features)
    print(str(model)+':')
    evaluate(test_labels, predicted_labels)

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error 

LogisticRegression():
Accuracy: 0.06946983546617916
Hamming Loss: 0.306346304518151
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.68      0.65       341
           1       0.34      0.30      0.32       186
           2       0.21      0.17      0.19       133
           3       0.19      0.17      0.18        81
           4       0.17      0.15      0.16        74
           5       0.10      0.12      0.11        72
           6       0.19      0.17      0.18        69

   micro avg       0.38      0.37      0.38       956
   macro avg       0.26      0.25      0.26       956
weighted avg       0.37      0.37      0.37       956
 samples avg       0.48      0.46      0.33       956

Confusion matrix:
 [[[ 71 135]
  [110 231]]

 [[254 107]
  [131  55]]

 [[330  84]
  [110  23]]

 [[406  60]
  [ 67  14]]

 [[419  54]
  [ 63  11]]

 [[394  81]
  [ 63   9]]

 [[427  51]
  [ 57  12]]]


/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error 

RandomForestClassifier(random_state=1):
Accuracy: 0.16270566727605118
Hamming Loss: 0.2230347349177331
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.92      0.74       341
           1       0.39      0.06      0.11       186
           2       0.11      0.01      0.01       133
           3       0.00      0.00      0.00        81
           4       0.33      0.01      0.03        74
           5       0.50      0.01      0.03        72
           6       0.50      0.01      0.03        69

   micro avg       0.59      0.34      0.44       956
   macro avg       0.35      0.15      0.14       956
weighted avg       0.41      0.34      0.29       956
 samples avg       0.64      0.45      0.41       956

Confusion matrix:
 [[[ 15 191]
  [ 28 313]]

 [[342  19]
  [174  12]]

 [[406   8]
  [132   1]]

 [[461   5]
  [ 81   0]]

 [[471   2]
  [ 73   1]]

 [[474   1]
  [ 71   1]]

 [[477   1]
  [ 68   1]]]


/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error 

SVC():
Accuracy: 0.17184643510054845
Hamming Loss: 0.2144162966832071
Classification Report:
               precision    recall  f1-score   support

           0       0.62      1.00      0.77       341
           1       1.00      0.00      0.00       186
           2       1.00      0.00      0.00       133
           3       1.00      0.00      0.00        81
           4       1.00      0.00      0.00        74
           5       1.00      0.00      0.00        72
           6       1.00      0.00      0.00        69

   micro avg       0.62      0.36      0.45       956
   macro avg       0.95      0.14      0.11       956
weighted avg       0.87      0.36      0.27       956
 samples avg       0.62      0.47      0.43       956

Confusion matrix:
 [[[  0 206]
  [  0 341]]

 [[361   0]
  [186   0]]

 [[414   0]
  [133   0]]

 [[466   0]
  [ 81   0]]

 [[473   0]
  [ 74   0]]

 [[475   0]
  [ 72   0]]

 [[478   0]
  [ 69   0]]]


### MLP

In [78]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def get_mlp(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

n_inputs, n_outputs = x.shape[1], y.shape[1]
mlp = get_mlp(n_inputs, n_outputs)
mlp.fit(train_features, train_labels, verbose=1, epochs=100)

2021-11-20 12:08:16.477558: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-20 12:08:16.483762: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 12:08:16.484054: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-11-20 12:08:16.484209: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-20 12:08:16.485310: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-20 12:08:16.486648: I tensorflow/stream_executor/platform/default/d

Epoch 1/100
 1/40 [..............................] - ETA: 0s - loss: 2.6225

2021-11-20 12:08:17.286345: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


40/40 [==============================] - 0s 2ms/step - loss: 0.8767
Epoch 2/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6815
Epoch 3/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6719
Epoch 4/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6624
Epoch 5/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6519
Epoch 6/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6427
Epoch 7/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6328
Epoch 8/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6244
Epoch 9/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6197
Epoch 10/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6123
Epoch 11/100
40/40 [==============================] - 0s 2ms/step - loss: 0.6022
Epoch 12/100
40/40 [==============================] - 0s 2ms/step - loss: 0.5888
Epoch 13/100
40/40 [=============================

In [79]:
predicted_labels_mlp = mlp.predict(test_features)
evaluate(test_labels, predicted_labels_mlp.round())

Accuracy: 0.170018281535649
Hamming Loss: 0.21467746147819275
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.99      0.77       341
           1       1.00      0.00      0.00       186
           2       1.00      0.00      0.00       133
           3       1.00      0.00      0.00        81
           4       1.00      0.00      0.00        74
           5       1.00      0.00      0.00        72
           6       1.00      0.00      0.00        69

   micro avg       0.62      0.35      0.45       956
   macro avg       0.95      0.14      0.11       956
weighted avg       0.87      0.35      0.27       956
 samples avg       0.63      0.47      0.43       956

Confusion matrix:
 [[[  3 203]
  [  4 337]]

 [[361   0]
  [186   0]]

 [[414   0]
  [133   0]]

 [[466   0]
  [ 81   0]]

 [[473   0]
  [ 74   0]]

 [[475   0]
  [ 72   0]]

 [[478   0]
  [ 69   0]]]


### Binary Relevance
ignores the possible correlations between class labels

In [80]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())
classifier.fit(test_features, test_labels)

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [81]:
predicted_labels_br = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_br)

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error 

Accuracy: 0.08957952468007313
Hamming Loss: 0.478453904413685
Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.38      0.50       341
           1       0.39      0.84      0.53       186
           2       0.31      0.85      0.45       133
           3       0.36      0.46      0.40        81
           4       0.18      0.89      0.30        74
           5       0.18      0.89      0.30        72
           6       0.18      0.94      0.30        69

   micro avg       0.30      0.66      0.41       956
   macro avg       0.33      0.75      0.40       956
weighted avg       0.45      0.66      0.45       956
 samples avg       0.35      0.68      0.40       956

Confusion matrix:
 [[[156  50]
  [210 131]]

 [[119 242]
  [ 30 156]]

 [[160 254]
  [ 20 113]]

 [[400  66]
  [ 44  37]]

 [[171 302]
  [  8  66]]

 [[185 290]
  [  8  64]]

 [[174 304]
  [  4  65]]]


/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


### Classfier Chains

In [82]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

classifier = ClassifierChain(LogisticRegression())
classifier.fit(train_features, train_labels)
# we should optimise this a little

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

ClassifierChain(classifier=LogisticRegression(), require_dense=[True, True])

In [83]:
predicted_labels_cc = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_cc)

Accuracy: 0.07312614259597806
Hamming Loss: 0.306346304518151
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.67      0.65       341
           1       0.34      0.30      0.32       186
           2       0.22      0.19      0.20       133
           3       0.18      0.15      0.16        81
           4       0.17      0.18      0.17        74
           5       0.14      0.17      0.15        72
           6       0.18      0.16      0.17        69

   micro avg       0.38      0.37      0.38       956
   macro avg       0.27      0.26      0.26       956
weighted avg       0.37      0.37      0.37       956
 samples avg       0.47      0.46      0.33       956

Confusion matrix:
 [[[ 71 135]
  [114 227]]

 [[254 107]
  [131  55]]

 [[325  89]
  [108  25]]

 [[411  55]
  [ 69  12]]

 [[411  62]
  [ 61  13]]

 [[400  75]
  [ 60  12]]

 [[429  49]
  [ 58  11]]]


### Label Powerset
takes correlations into account!

In [84]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(LogisticRegression())
classifier.fit(train_features, train_labels)

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LabelPowerset(classifier=LogisticRegression(), require_dense=[True, True])

In [85]:
predicted_labels_lp = classifier.predict(test_features)
evaluate(test_labels, predicted_labels_lp)

Accuracy: 0.06764168190127971
Hamming Loss: 0.30373465656829457
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.62      0.62       341
           1       0.39      0.36      0.37       186
           2       0.22      0.19      0.20       133
           3       0.14      0.10      0.12        81
           4       0.16      0.18      0.17        74
           5       0.11      0.11      0.11        72
           6       0.09      0.06      0.07        69

   micro avg       0.38      0.35      0.37       956
   macro avg       0.25      0.23      0.24       956
weighted avg       0.37      0.35      0.36       956
 samples avg       0.49      0.43      0.31       956

Confusion matrix:
 [[[ 80 126]
  [130 211]]

 [[255 106]
  [119  67]]

 [[327  87]
  [108  25]]

 [[418  48]
  [ 73   8]]

 [[406  67]
  [ 61  13]]

 [[407  68]
  [ 64   8]]

 [[437  41]
  [ 65   4]]]


/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:1675: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


### Multi Label KNN

In [ ]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

mlknn = MLkNN(k=10)

x_train = lil_matrix(train_features).toarray()
y_train = lil_matrix(train_labels).toarray()
x_test = lil_matrix(test_features).toarray()
# train
mlknn.fit(x_train, y_train)
# predict
# predictions_new = classifier_new.predict(x_test)
# # accuracy
# print("Accuracy = ",accuracy_score(y_test,predictions_new))
# print("\n")
predicted_labels_mlknn = mlknn.predict(x_test)
evaluate(test_labels, predicted_labels_mlknn)